In [9]:
import mne
mne.set_log_level('error')
import matplotlib
import matplotlib.pyplot as plt
from functions import *
from autoreject import get_rejection_threshold
matplotlib.use('TkAgg')

### Load the data

In [10]:
data_path = "/Users\sunnivlf\Documents/Data Set Alcohol Detector/Subjects" # Load this from where you have saved it on your computer
p_id = "/" + "sub-P019" + "/" + "ses-S002" + "/" + "sub-P019_ses-S002_task-Default_run-005"
raw = mne.io.read_raw_fif(data_path + "/" + p_id + "_eeg.fif", preload=True)

In [11]:
# Filter settings
ica_low_cut = 1.0       # ICA is sensitive to low-frequency drifts and therefore requires the data to be high-pass filtered prior to fitting. Typically, a cutoff frequency of 1 Hz is recommended.
ica_hi_cut  = 30

raw_ica = raw.copy().filter(ica_low_cut, ica_hi_cut)

### Segment data for ICA

In [12]:
# Break raw data into 1s epochs
tstep = 1.0
events_ica = mne.make_fixed_length_events(raw_ica, duration=tstep)
epochs_ica = mne.Epochs(raw_ica, events_ica,
                        tmin=0.0, tmax=tstep,
                        baseline=None,
                        preload=True)

reject = get_rejection_threshold(epochs_ica);
reject

Estimating rejection dictionary for eeg


{'eeg': 0.00026890460696842445}

### Fit ICA to data

In [5]:
# ICA parameters
random_state = 42   # ensures ICA is reproducable each time it's run
ica_n_components = .99     # Specify n_components as a decimal to set % explained variance

# Fit ICA
ica = mne.preprocessing.ICA(n_components=ica_n_components,
                            random_state=random_state,
                            )
ica.fit(epochs_ica,
        reject=reject,
        tstep=tstep)

Method,fastica
Fit,27 iterations on epochs (108181 samples)
ICA components,7
Available PCA components,16
Channel types,eeg
ICA components marked for exclusion,—


### Visualiza ICA components

In [6]:
ica.plot_components();

### Identify EOG artifacts from ICA components

In [15]:
ica_z_thresh = 1.96 
eog_indices, eog_scores = ica.find_bads_eog(raw_ica, 
                                            ch_name=['Fp2'], 
                                            threshold=ica_z_thresh)
ica.exclude = eog_indices
ica.plot_scores(eog_scores);
# ica.apply(raw.copy())

<Raw | sub-P019_ses-S002_task-Default_run-005_eeg.fif, 17 x 107797 (431.2 s), ~14.0 MB, data loaded>

### Save the ICA

In [14]:
data_path = "/Users\sunnivlf\Documents/Preprocessed data" ## Save this somewhere on your own computer
ica.save(data_path + '/' + p_id + '-ica.fif', 
        overwrite=True);